In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### QA Pair 를 생성할 PDF 를 로드합니다.

In [2]:
from unstructured.partition.pdf import partition_pdf


def extract_pdf_elements(filepath):
    """
    PDF 파일에서 이미지, 테이블, 그리고 텍스트 조각을 추출합니다.
    path: 이미지(.jpg)를 저장할 파일 경로
    fname: 파일 이름
    """
    return partition_pdf(
        filename=filepath,
        extract_images_in_pdf=False,  # PDF 내 이미지 추출 활성화
        infer_table_structure=False,  # 테이블 구조 추론 활성화
        chunking_strategy="by_title",  # 제목별로 텍스트 조각화
        max_characters=4000,  # 최대 문자 수
        new_after_n_chars=3800,  # 이 문자 수 이후에 새로운 조각 생성
        combine_text_under_n_chars=2000,  # 이 문자 수 이하의 텍스트는 결합
    )

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# PDF 파일 로드
elements = extract_pdf_elements("../12-RAG/data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf")

In [5]:
# 로드한 TEXT 청크 수
len(elements)

92

In [6]:
print(elements[2])

평했다.

‘열일곱 살부터 논어를 읽었는데, 그때에도 이미 문장의 뜻은 알

고 있었다. 그러나 논어를 오래도록 읽을수록 그 뜻이 더욱 의미심

14 14

장해짐을 느꼈다.’

정자는 《논어》를 읽고 난 뒤 네 부류의 사람이 나타난다고 했다.

‘첫째, 아무런 감흥을 느끼지 못하는 사람이 있다. 둘째, 그중 한

두 구절을 얻고 기뻐하는 사람이 있다. 셋째, 그것을 좋아할 줄 알게

되는 사람이 있다. 넷째, 덩실덩실 손발이 절로 들썩이는 것도 모르

는 사람이 있다.’

15 15

성인 공자

자 왈 子

십 실 지 읍

曰 十

室之邑

필 유 충 신 必

여 구 자 언

불 여 구 지 호 학 야

有忠信 如

丘者焉 不

如丘之好學也

공자가 말했다.

열 집쯤 되는 작은 마을에도 나만큼 충직하고 믿을 만한 사람은 꼭 있을 것이다.

그러나 나만큼 배우기를 좋아하는 사람은 없을 것이다.

_공야장 公冶長 27

論 語

공자는 세 살 때 아버지를 여의었다. 그 때문에 그의 어머니가 홀

로 집안 살림을 꾸려나가야 했는데, 자연히 매우 가난했다. 그러다

보니 글방에 다니며 공부하는 것은 언감생심이었다. 어떡하든 집안

을 도와야 했기에 그는 어린 나이에도 공사장에 나가 돌을 나르거

나 목공의 조수 노릇을 하며 돈을 벌었다.

하지만 그 불우한 환경에서도 공자는 학문을 포기하지 않았다.

일을 마치고 귀가할라치면 그는 녹초가 된 몸을 또다시 바로 세우

며 늦은 밤까지 책을 읽었다.

16

‘나만큼 배우기를 좋아하는 사람은 없을 것이다’라는 공자의 말

을 통해, 우리는 그가 당시 학문에 얼마나 큰 자부심을 가졌었는지

알 수 있다. 분명 그는 대단한 노력가였다. 그는 길을 가는 행인 세

명 중에도 스승이 있다고 말할 정도로 누구에게든 배웠다. 선한 사

람을 본받아 더욱 선해지려 노력했고, 악한 사람을 반면교사로 삼

아 악해지지 않으려 부단히 수양했다.

결국 공자는 사람으로서 이를 수 있는 최고의 경지에 올랐으니,

예수·석가·소크라테스와 함

### QA Pair 생성

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


# 원하는 데이터 구조를 정의합니다.
class QAPair(BaseModel):
    question: str = Field(description="Question generated from the text")
    answer: str = Field(description="Answer related to the question")

In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate only questions based on the below query.
You are an Teacher/Professor in {domain}. 
Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
You are not to provide more or less than this number of questions. 
The question(s) should be diverse in nature across the document. 
The purpose of question(s) is to test the understanding of the students on the context information provided.
You must also provide the answer to each question. The answer should be based on the context information provided only.

Restrict the question(s) to the context information provided only.
QUESTION and ANSWER should be written in Korean. response in JSON format which contains the `question` and `answer`.
ANSWER should be a complete sentence.

#Format:
```json
{{
    "QUESTION": "바이든 대통령이 서명한 '안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령'의 주요 목적 중 하나는 무엇입니까?",
    "ANSWER": "바이든 대통령이 서명한 행정명령의 주요 목적은 AI의 안전 마련과 보안 기준 마련을 위함입니다."
}},
{{
    "QUESTION": "메타의 라마2가 오픈소스 모델 중에서 어떤 유형의 작업에서 가장 우수한 성능을 발휘했습니까?",
    "ANSWER": "메타의 라마2는 RAG 없는 질문과 답변 및 긴 형식의 텍스트 생성에서 오픈소스 모델 중 가장 우수한 성능을 발휘했습니다."    
}},
{{
    "QUESTION": "IDC 예측에 따르면 2027년까지 생성 AI 플랫폼과 애플리케이션 시장의 매출은 얼마로 전망되나요?",
    "ANSWER": "IDC 예측에 따르면 2027년까지 생성 AI 플랫폼과 애플리케이션 시장의 매출은 283억 달러로 전망됩니다."    
}}
```
"""
)

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=QAPair)

chain = (
    prompt
    | ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    | parser
)  # 체인을 구성합니다.

qa_pair = []

for element in elements[1:]:
    if element.text:
        qa_pair.append(
            chain.invoke(
                {"context": element.text, "domain": "AI", "num_questions": "3"}
            )
        )

```json
{
    "QUESTION": "《논어》는 어떤 책이며, 그 책의 주요 내용은 무엇입니까?",
    "ANSWER": "《논어》는 유가의 성전으로, 유교의 근본 사상을 다루고 있습니다. 이 책에는 공자가 생전에 제자, 관료들과 교감하면서 드러낸 사상과 언행이 압축적으로 수록되어 있습니다."
},
{
    "QUESTION": "공자의 핵심 사상은 무엇이며, 정치에서는 어떤 가르침을 주었습니까?",
    "ANSWER": "공자의 핵심 사상은 인과 예입니다. 정치에서는 덕으로써 백성을 다스리며, 정을 기본으로 하여 모범을 보이라고 가르쳤습니다."
},
{
    "QUESTION": "《논어》의 구성은 어떻게 되어 있으며, 각 편의 제목은 주로 어떻게 붙여졌습니까?",
    "ANSWER": "《논어》는 총 20편으로 구성되어 있으며, 각 편의 제목은 주로 맨 처음의 2자 또는 3자를 따서 붙였습니다."
}
``````json
{
    "QUESTION": "공자가 '열 집쯤 되는 작은 마을에도 나만큼 충직하고 믿을 만한 사람은 꼭 있을 것이다. 그러나 나만큼 배우기를 좋아하는 사람은 없을 것이다.'라고 말한 이유는 무엇입니까?",
    "ANSWER": "공자는 자신이 학문에 대한 큰 자부심을 가지고 있었고, 누구보다도 배우기를 좋아했기 때문에 이렇게 말했습니다."
},
{
    "QUESTION": "공자가 학문을 대하는 자세에서 가장 중시한 것은 무엇이었습니까?",
    "ANSWER": "공자가 학문을 대하는 자세에서 가장 중시한 것은 '옛것을 열심히 익혀서 그것을 미루어서 새것을 아는 것', 즉 '온고지신'이었습니다."
},
{
    "QUESTION": "공자가 '세 사람이 함께 걸어가고 있다면 그중에 나의 스승이 반드시 있게 마련이다.'라고 말한 이유는 무엇입니까?",
    "ANSWER": "공자는 매사 받아들이려는 열린 자세를 취했기 때문에 잘났든 못났든 누구나 공자의 스승이 될 수 있다고 생각했기 때문입니다."

In [10]:
qa_pair[8]

{'QUESTION': '공자가 스스로 정한 네 가지 금기 사항은 무엇입니까?',
 'ANSWER': '공자가 스스로 정한 네 가지 금기 사항은 억측을 부리는 일, 어떤 사안을 두고 장담하는 일, 자기 의견만을 고집부리는 일, 이기적인 일입니다.'}

In [11]:
qa_pair

[{'QUESTION': '《논어》는 어떤 책이며, 그 책의 주요 내용은 무엇입니까?',
  'ANSWER': '《논어》는 유가의 성전으로, 유교의 근본 사상을 다루고 있습니다. 이 책에는 공자가 생전에 제자, 관료들과 교감하면서 드러낸 사상과 언행이 압축적으로 수록되어 있습니다.'},
 {'QUESTION': "공자가 '열 집쯤 되는 작은 마을에도 나만큼 충직하고 믿을 만한 사람은 꼭 있을 것이다. 그러나 나만큼 배우기를 좋아하는 사람은 없을 것이다.'라고 말한 이유는 무엇입니까?",
  'ANSWER': '공자는 자신이 학문에 대한 큰 자부심을 가지고 있었고, 누구보다도 배우기를 좋아했기 때문에 이렇게 말했습니다.'},
 {'QUESTION': '공자가 자공에게 무덤을 가리키며 전하고자 한 무언의 가르침은 무엇입니까?',
  'ANSWER': '공자가 무덤을 가리킨 까닭은, 죽는 순간까지 학문을 계속해야 한다는 무언의 가르침을 주기 위해서였습니다.'},
 {'QUESTION': '공자가 거친 밥에 물을 마시고 팔을 굽혀 베개를 삼더라도 즐거움이 있다고 말한 이유는 무엇입니까?',
  'ANSWER': '공자는 의롭지 않은 방법으로 얻은 부귀는 나에게 뜬구름 같은 것이라고 생각했기 때문에, 거친 밥에 물을 마시고 팔을 굽혀 베개를 삼더라도 그 가운데 즐거움이 있다고 말했습니다.'},
 {'QUESTION': "공자가 제자 자공에게 '너는 그릇 같은 존재'라고 말했을 때, 자공이 다시 물어본 질문은 무엇이었습니까?",
  'ANSWER': "자공이 다시 물어본 질문은 '어떤 그릇을 말씀하시는지 자세히 들려주십시오'였습니다."},
 {'QUESTION': '자로는 공자의 가르침을 어떻게 받아들였으며, 그의 학습 태도는 어떠했습니까?',
  'ANSWER': '자로는 공자의 가르침을 무조건 받아들였으며, 아무리 하찮은 말씀이라도 꼭 기록해두고 반복해서 그 뜻을 음미하곤 했습니다. 비록 이해력이 뛰어나지는 않았지만, 한 번 들은 것은 꼭 기록해두었다가 반복 학습으로

In [12]:
# 디버깅을 위한 데이터셋 추가
additional_qa = [
    {
        "QUESTION": "테디노트 유튜브 채널에 대해서 알려주세요.",
        "ANSWER": "테디노트(TeddyNote)는 데이터 분석, 머신러닝, 딥러닝 등의 주제를 다루는 유튜브 채널입니다. 이 채널을 운영하는 이경록님은 데이터 분석과 인공지능에 대한 다양한 강의를 제공하며, 초보자도 쉽게 따라할 수 있도록 친절하게 설명합니다.",
    },
    {
        "QUESTION": "랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?",
        "ANSWER": "테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314",
    },
    {
        "QUESTION": "테디노트 운영자에 대해서 알려주세요",
        "ANSWER": "테디노트(TeddyNote) 운영자는 이경록(Teddy Lee)입니다. 그는 데이터 분석, 머신러닝, 딥러닝 분야에서 활동하는 전문가로, 다양한 교육 및 강의를 통해 지식을 공유하고 있습니다. 이경록님은 여러 기업과 교육기관에서 파이썬, 데이터 분석, 텐서플로우 등 다양한 주제로 강의를 진행해 왔습니다",
    },
]

In [19]:
qa_pair.extend(additional_qa)
original_qa = qa_pair

### jsonl 파일로 저장

In [20]:
original_qa

[{'QUESTION': '《논어》는 어떤 책이며, 그 책의 주요 내용은 무엇입니까?',
  'ANSWER': '《논어》는 유가의 성전으로, 유교의 근본 사상을 다루고 있습니다. 이 책에는 공자가 생전에 제자, 관료들과 교감하면서 드러낸 사상과 언행이 압축적으로 수록되어 있습니다.'},
 {'QUESTION': "공자가 '열 집쯤 되는 작은 마을에도 나만큼 충직하고 믿을 만한 사람은 꼭 있을 것이다. 그러나 나만큼 배우기를 좋아하는 사람은 없을 것이다.'라고 말한 이유는 무엇입니까?",
  'ANSWER': '공자는 자신이 학문에 대한 큰 자부심을 가지고 있었고, 누구보다도 배우기를 좋아했기 때문에 이렇게 말했습니다.'},
 {'QUESTION': '공자가 자공에게 무덤을 가리키며 전하고자 한 무언의 가르침은 무엇입니까?',
  'ANSWER': '공자가 무덤을 가리킨 까닭은, 죽는 순간까지 학문을 계속해야 한다는 무언의 가르침을 주기 위해서였습니다.'},
 {'QUESTION': '공자가 거친 밥에 물을 마시고 팔을 굽혀 베개를 삼더라도 즐거움이 있다고 말한 이유는 무엇입니까?',
  'ANSWER': '공자는 의롭지 않은 방법으로 얻은 부귀는 나에게 뜬구름 같은 것이라고 생각했기 때문에, 거친 밥에 물을 마시고 팔을 굽혀 베개를 삼더라도 그 가운데 즐거움이 있다고 말했습니다.'},
 {'QUESTION': "공자가 제자 자공에게 '너는 그릇 같은 존재'라고 말했을 때, 자공이 다시 물어본 질문은 무엇이었습니까?",
  'ANSWER': "자공이 다시 물어본 질문은 '어떤 그릇을 말씀하시는지 자세히 들려주십시오'였습니다."},
 {'QUESTION': '자로는 공자의 가르침을 어떻게 받아들였으며, 그의 학습 태도는 어떠했습니까?',
  'ANSWER': '자로는 공자의 가르침을 무조건 받아들였으며, 아무리 하찮은 말씀이라도 꼭 기록해두고 반복해서 그 뜻을 음미하곤 했습니다. 비록 이해력이 뛰어나지는 않았지만, 한 번 들은 것은 꼭 기록해두었다가 반복 학습으로

In [16]:
import json

In [21]:
with open("data/qa_pair.jsons", "w") as f:
    json.dumps(original_qa)

In [22]:
with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
    original_qa = json.dumps(qa_pair, ensure_ascii=False)
    
# 디버깅을 위한 데이터셋 추가
additional_qa = [
    {
        "QUESTION": "여러분의 이름을 넣어보세요",
        "ANSWER": "답변을 적어보세요...",
    },
    {
        "QUESTION": "랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?",
        "ANSWER": "테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314",
    },
    {
        "QUESTION": "테디노트 운영자에 대해서 알려주세요",
        "ANSWER": "테디노트(TeddyNote) 운영자는 이경록(Teddy Lee)입니다. 그는 데이터 분석, 머신러닝, 딥러닝 분야에서 활동하는 전문가로, 다양한 교육 및 강의를 통해 지식을 공유하고 있습니다. 이경록님은 여러 기업과 교육기관에서 파이썬, 데이터 분석, 텐서플로우 등 다양한 주제로 강의를 진행해 왔습니다",
    },
]

original_qa = json.loads(original_qa)
original_qa.extend(additional_qa)
original_qa

[{'QUESTION': '《논어》는 어떤 책이며, 그 책의 주요 내용은 무엇입니까?',
  'ANSWER': '《논어》는 유가의 성전으로, 유교의 근본 사상을 다루고 있습니다. 이 책에는 공자가 생전에 제자, 관료들과 교감하면서 드러낸 사상과 언행이 압축적으로 수록되어 있습니다.'},
 {'QUESTION': "공자가 '열 집쯤 되는 작은 마을에도 나만큼 충직하고 믿을 만한 사람은 꼭 있을 것이다. 그러나 나만큼 배우기를 좋아하는 사람은 없을 것이다.'라고 말한 이유는 무엇입니까?",
  'ANSWER': '공자는 자신이 학문에 대한 큰 자부심을 가지고 있었고, 누구보다도 배우기를 좋아했기 때문에 이렇게 말했습니다.'},
 {'QUESTION': '공자가 자공에게 무덤을 가리키며 전하고자 한 무언의 가르침은 무엇입니까?',
  'ANSWER': '공자가 무덤을 가리킨 까닭은, 죽는 순간까지 학문을 계속해야 한다는 무언의 가르침을 주기 위해서였습니다.'},
 {'QUESTION': '공자가 거친 밥에 물을 마시고 팔을 굽혀 베개를 삼더라도 즐거움이 있다고 말한 이유는 무엇입니까?',
  'ANSWER': '공자는 의롭지 않은 방법으로 얻은 부귀는 나에게 뜬구름 같은 것이라고 생각했기 때문에, 거친 밥에 물을 마시고 팔을 굽혀 베개를 삼더라도 그 가운데 즐거움이 있다고 말했습니다.'},
 {'QUESTION': "공자가 제자 자공에게 '너는 그릇 같은 존재'라고 말했을 때, 자공이 다시 물어본 질문은 무엇이었습니까?",
  'ANSWER': "자공이 다시 물어본 질문은 '어떤 그릇을 말씀하시는지 자세히 들려주십시오'였습니다."},
 {'QUESTION': '자로는 공자의 가르침을 어떻게 받아들였으며, 그의 학습 태도는 어떠했습니까?',
  'ANSWER': '자로는 공자의 가르침을 무조건 받아들였으며, 아무리 하찮은 말씀이라도 꼭 기록해두고 반복해서 그 뜻을 음미하곤 했습니다. 비록 이해력이 뛰어나지는 않았지만, 한 번 들은 것은 꼭 기록해두었다가 반복 학습으로

In [23]:
with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
    for qa in original_qa:
        f.write(json.dumps(qa, ensure_ascii=False) + "\n")

In [25]:
with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
    for qa in qa_pair:
        if isinstance(qa, dict) and "QUESTION" in qa and "ANSWER" in qa:
            qa_modified = {
                "instruction": qa["QUESTION"],
                "input": "",
                "output": qa["ANSWER"],
            }
            f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")
        else:
            print(f"Invalid QA pair: {qa}")

Invalid QA pair: [{'QUESTION': '사단이 발전하면 무엇이 되며, 인간의 본성은 무엇이라고 결론지을 수 있습니까?', 'ANSWER': '사단이 발전하면 선이 되며, 인간의 본성은 선한 것이라고 결론지을 수 있습니다.'}, {'QUESTION': '맹자가 제나라 선왕에게 나라가 잘 다스려지지 않았을 때 어떻게 처리할 것인지 물었을 때, 선왕의 반응은 어땠습니까?', 'ANSWER': '맹자의 질문에 선왕은 공연히 좌우를 두리번거리며 말끝을 흐렸습니다.'}, {'QUESTION': '맹자가 인이 불인을 이기는 것을 무엇에 비유했으며, 지금 인을 행하는 사람들의 기대를 무엇에 비유했습니까?', 'ANSWER': '맹자는 인이 불인을 이기는 것을 물이 불을 이기는 것에 비유했으며, 지금 인을 행하는 사람들의 기대를 한 잔의 물로 한 수레의 나무에 붙은 불을 끄려는 욕심에 비유했습니다.'}]
Invalid QA pair: [{'QUESTION': '군자가 남을 가르치는 다섯 가지 방법 중 첫 번째 방법은 무엇인가요?', 'ANSWER': '첫 번째 방법은 제때에 내린 비로 인해 초목이 저절로 무성하게 자라듯이 스스로 계발하도록 도와주는 방법입니다.'}, {'QUESTION': '맹자가 고자에게 산골짜기 오솔길을 비유하여 말한 내용은 무엇인가요?', 'ANSWER': '맹자는 고자에게 산골짜기 오솔길도 사람이 다니면 길이 나지만 사람 발길이 조금 뜸하면 길은 없어지고 풀로 덮여버린다고 말했습니다.'}, {'QUESTION': "맹자가 말한 '민 위 귀, 사 직 차 지, 군 위 경'의 의미는 무엇인가요?", 'ANSWER': "맹자가 말한 '민 위 귀, 사 직 차 지, 군 위 경'의 의미는 나라에서 백성이 가장 귀중하고, 사직이 그다음이며, 임금이 가장 가벼운 존재라는 것입니다."}]


In [26]:
from datasets import load_dataset

# JSONL 파일 경로
jsonl_file = "qa_pair.jsonl"

# JSONL 파일을 Dataset으로 로드
dataset = load_dataset("json", data_files=jsonl_file)

Generating train split: 95 examples [00:00, 5251.31 examples/s]


In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 95
    })
})

In [29]:
# from huggingface_hub import HfApi

# # HfApi 인스턴스 생성
# api = HfApi()

# # 데이터셋을 업로드할 리포지토리 이름
# repo_name = "teddylee777/QA-Dataset-mini"

# # 데이터셋을 허브에 푸시
# dataset.push_to_hub(repo_name, token="허깅페이스 토큰")